In [25]:
import mlflow
from mlflow import MlflowClient
from mlflow.entities import ViewType


In [2]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
EXPERIMENT_NAME = 'portugal-rent-price'

In [3]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [7]:
query = 'tags.runtype = "hyperparameter_tuning"'

In [21]:
model_tunnings = client.search_runs(
            experiment_ids=["1"],
            filter_string=query,
            order_by=["metrics.rmse ASC", "metrics.r2 DESC"],
            max_results=5,
        )

In [22]:
model_tunnings[0]

<Run: data=<RunData: metrics={'r2': 0.6988719204367879, 'rmse': 1104.0525084883911}, params={'learning_rate': '0.0988686395515279',
 'max_depth': '5',
 'min_child_weight': '3.3709971786192416',
 'objective': 'reg:squarederror',
 'reg_alpha': '0.11891552093518695',
 'reg_lambda': '0.13849211277084655',
 'seed': '42'}, tags={'mlflow.runName': 'likeable-conch-145',
 'mlflow.source.git.commit': '31ff8daf350dc8c1b55b5a9e93ca5b52824d532f',
 'mlflow.source.name': '.\\src\\training\\train.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'MarcosPaulo',
 'model': 'xboostRegressor',
 'runtype': 'hyperparameter_tuning'}>, info=<RunInfo: artifact_uri='s3://final-project-mlops/airflow/1/bf188ed76cb04c5b9a5825cafd5811bb/artifacts', end_time=1690130235386, experiment_id='1', lifecycle_stage='active', run_id='bf188ed76cb04c5b9a5825cafd5811bb', run_name='likeable-conch-145', run_uuid='bf188ed76cb04c5b9a5825cafd5811bb', start_time=1690130235217, status='FINISHED', user_id='MarcosPaulo'>, inputs=<RunI

In [12]:
best_param = model_tunnings[0].data.params

In [13]:
best_param

{'learning_rate': '0.0988686395515279',
 'max_depth': '5',
 'min_child_weight': '3.3709971786192416',
 'objective': 'reg:squarederror',
 'reg_alpha': '0.11891552093518695',
 'reg_lambda': '0.13849211277084655',
 'seed': '42'}

In [26]:
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
query = 'tags.runtype = "best_model"'
best_run = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    filter_string=query,
    order_by=["metrics.rmse ASC"],
)[0]

In [27]:
best_run

<Run: data=<RunData: metrics={'rmse': 2842.0227587570203}, params={'learning_rate': '0.010223292622345987',
 'max_depth': '3',
 'min_child_weight': '0.8738004629860953',
 'objective': 'reg:squarederror',
 'reg_alpha': '0.007200362321984579',
 'reg_lambda': '0.19740775735980534',
 'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "b59f734d975840a5ba5198d21b6b4e83", '
                             '"artifact_path": "models", "utc_time_created": '
                             '"2023-07-23 16:38:47.606025", "flavors": '
                             '{"python_function": {"loader_module": '
                             '"mlflow.xgboost", "python_version": "3.9.12", '
                             '"data": "model.xgb", "env": {"conda": '
                             '"conda.yaml", "virtualenv": "python_env.yaml"}}, '
                             '"xgboost": {"xgb_version": "1.7.6", "data": '
                             '"model.xgb", "model_class": '
                             '"x

In [5]:
model_name = EXPERIMENT_NAME
latest_versions = client.get_latest_versions(model_name)

In [6]:
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")
    print(version)


version: 1, stage: Production
<ModelVersion: aliases=[], creation_timestamp=1689533803895, current_stage='Production', description='The model version 1 was transitioned to Production on 2023-07-16', last_updated_timestamp=1689533803987, name='portugal-rent-price', run_id='3d9d5a71529743d9b2f52c86fc657510', run_link='', source='s3://final-project-mlops/airflow/1/3d9d5a71529743d9b2f52c86fc657510/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>
version: 5, stage: Archived
<ModelVersion: aliases=[], creation_timestamp=1690130329846, current_stage='Archived', description='The model version 5 was transitioned to Archived on 2023-07-23', last_updated_timestamp=1690130329971, name='portugal-rent-price', run_id='3d9d5a71529743d9b2f52c86fc657510', run_link='', source='s3://final-project-mlops/airflow/1/3d9d5a71529743d9b2f52c86fc657510/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='5'>


In [12]:
model_prod_id = [version.run_id for version in latest_versions if version.current_stage == "Production"]

In [13]:
model_prod_id

['9eaf128191ec4d72985f202969b34b88']

In [15]:
prod_model = client.get_run(model_prod_id[0])

In [21]:
prod_model.data.metrics['rmse']

34559.642613140764